作業要求：<br>

請使用10和11月資料當作訓練集，12月之資料當作測試集，<br>

將前六小時的汙染物數據做為特徵，未來第一個小時/未來第六個小時的pm2.5數據為預測目標<br>

使用兩種模型 Linear Regression 和 XGBoost 建模並計算MAE<br>

## 載入套件

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

## 讀取資料


In [2]:
data = pd.read_csv('新竹_2020.csv', encoding='Big5')

data = data[1:]  # 一開始第一行是無資料的row
# 去除資料中的空白  
def rstrip(input_data):
    return [data.rstrip() for data in input_data]
# 移除columns 及 data中多餘的空格
data.columns = rstrip(data.columns)
for i in range(len(data)):
    data.iloc[i] = rstrip(data.iloc[i])
data


,測站,日期,測項,00,01,02,03,04,05,06,...,14,15,16,17,18,19,20,21,22,23
1,新竹,2020/01/01 00:00:00,AMB_TEMP,15.2,15.2,15.3,15.3,15.3,15.4,15.5,...,18.1,18.2,17.9,17.3,16.7,16.4,16.2,16.1,16,15.8
2,新竹,2020/01/01 00:00:00,CH4,1.74,1.74,1.77,1.78,1.77,1.77,1.77,...,1.78,1.78,1.77,1.8,1.81,1.82,1.85,1.83,1.92,1.94
3,新竹,2020/01/01 00:00:00,CO,0.28,0.25,0.24,0.22,0.2,0.19,0.2,...,0.28,0.29,0.28,0.34,0.39,0.41,0.46,0.49,0.58,0.52
4,新竹,2020/01/01 00:00:00,NMHC,0.06,0.07,0.05,0.05,0.05,0.05,0.07,...,0.09,0.09,0.07,0.08,0.12,0.12,0.16,0.14,0.17,0.2
5,新竹,2020/01/01 00:00:00,NO,0.3,0.6,0.6,0.6,0.3,0.3,0.5,...,1.6,1.6,1.2,0.7,0.9,1.1,1.1,1.7,1.8,1.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6584,新竹,2020/12/31 00:00:00,THC,2.01,2.02,2,2,1.99,2,1.98,...,2.03,2.07,2.07,2.1,2.1,2.07,2.07,2.05,2.04,2.07
6585,新竹,2020/12/31 00:00:00,WD_HR,54,55,54,53,58,52,52,...,54,50,52,45,47,42,42,47,45,44
6586,新竹,2020/12/31 00:00:00,WIND_DIREC,53,52,57,58,49,54,36,...,48,43,44,33,50,40,46,46,51,38
6587,新竹,2020/12/31 00:00:00,WIND_SPEED,4.7,4.6,4.7,4.9,4.1,5.3,5.5,...,4.5,4.4,4.2,3.8,3.7,4.7,4.5,4.4,3.9,3.9


## 資料前處理
### (a)取出10.11.12月資料


In [3]:
date = data['日期'] >= '2020/10/01'
data = pd.DataFrame(data[date])
data['日期'] = pd.to_datetime(data['日期'])
data

,測站,日期,測項,00,01,02,03,04,05,06,...,14,15,16,17,18,19,20,21,22,23
4933,新竹,2020-10-01,AMB_TEMP,23.7,23.8,23.8,23.9,23.9,23.8,24.1,...,29.9,29.6,28.7,27.5,26.4,25.7,25.5,25.3,24.9,24.5
4934,新竹,2020-10-01,CH4,1.97,1.95,1.96,1.96,1.95,1.96,1.97,...,1.97,1.98,1.97,2,2.03,2.04,2.05,2.02,2.1,2.14
4935,新竹,2020-10-01,CO,0.23,0.22,0.21,0.2,0.2,0.22,0.24,...,0.29,0.3,0.33,0.38,0.46,0.5,0.45,0.39,0.46,0.45
4936,新竹,2020-10-01,NMHC,0.06,0.05,0.03,0.03,0.03,0.04,0.04,...,0.06,0.07,0.09,0.11,0.13,0.15,0.1,0.07,0.12,0.18
4937,新竹,2020-10-01,NO,1.2,0.7,0.5,0.7,0.5,0.3,0.7,...,1.3,1,0.9,0.8,0.5,0.9,0.9,0.3,0.7,0.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6584,新竹,2020-12-31,THC,2.01,2.02,2,2,1.99,2,1.98,...,2.03,2.07,2.07,2.1,2.1,2.07,2.07,2.05,2.04,2.07
6585,新竹,2020-12-31,WD_HR,54,55,54,53,58,52,52,...,54,50,52,45,47,42,42,47,45,44
6586,新竹,2020-12-31,WIND_DIREC,53,52,57,58,49,54,36,...,48,43,44,33,50,40,46,46,51,38
6587,新竹,2020-12-31,WIND_SPEED,4.7,4.6,4.7,4.9,4.1,5.3,5.5,...,4.5,4.4,4.2,3.8,3.7,4.7,4.5,4.4,3.9,3.9


## (b)將資料切割成訓練集(10.11月)以及測試集(12月)


In [4]:
train_data = data[data['日期'] < '2020/12/01']
test_data = data[data['日期'] >= '2020/12/01']

In [5]:
test_data

,測站,日期,測項,00,01,02,03,04,05,06,...,14,15,16,17,18,19,20,21,22,23
6031,新竹,2020-12-01,AMB_TEMP,18.5,18.4,18.3,18.2,18.2,18.3,18.4,...,21.4,20.7,20.1,19.8,19.7,19.9,20.5,20.6,20.5,20.3
6032,新竹,2020-12-01,CH4,1.95,1.95,1.95,1.95,1.95,1.95,1.94,...,1.92,1.89,1.94,1.95,1.96,1.94,1.91,1.91,1.91,1.92
6033,新竹,2020-12-01,CO,0.17,0.16,0.16,0.16,0.16,0.17,0.17,...,0.22,0.2,0.25,0.33,0.37,0.32,0.25,0.22,0.21,0.2
6034,新竹,2020-12-01,NMHC,0.06,0.05,0.06,0.06,0.04,0.06,0.03,...,0.07,0.08,0.11,0.15,0.16,0.12,0.06,0.04,0.04,0.05
6035,新竹,2020-12-01,NO,1.3,1.5,1.2,1.3,1.2,1.3,1.3,...,#,#,2.3,1.9,1.7,1.6,1.4,1.2,1,1.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6584,新竹,2020-12-31,THC,2.01,2.02,2,2,1.99,2,1.98,...,2.03,2.07,2.07,2.1,2.1,2.07,2.07,2.05,2.04,2.07
6585,新竹,2020-12-31,WD_HR,54,55,54,53,58,52,52,...,54,50,52,45,47,42,42,47,45,44
6586,新竹,2020-12-31,WIND_DIREC,53,52,57,58,49,54,36,...,48,43,44,33,50,40,46,46,51,38
6587,新竹,2020-12-31,WIND_SPEED,4.7,4.6,4.7,4.9,4.1,5.3,5.5,...,4.5,4.4,4.2,3.8,3.7,4.7,4.5,4.4,3.9,3.9


## (c)缺失值以及無效值以前後一小時平均值取代(如果前一小時仍有空值，再取更前一小時)

In [6]:
# 先對資料做轉置以利資料做後續處理
# 宣告一個新的 DataFrame 來儲存合併後的資料
train = pd.DataFrame()
test = pd.DataFrame()


for head in train_data['測項'].unique():
    train[head] = np.array(train_data[train_data['測項'] == head].iloc[:,3:]).ravel()
    test[head] = np.array(test_data[test_data['測項'] == head].iloc[:,3:]).ravel()

In [7]:
#缺失值處理

# 建立function 偵測無效字元
# 無效字元，以空白取代(np.nan)
invalid_symbol = ('#', '*', 'x', 'A')


def InvalidConvertor(df):
    for row, record in df.iterrows():
        for column, record in enumerate(record):
            if record.endswith(invalid_symbol):
                df.iloc[row, column] = np.nan
    return df


train = InvalidConvertor(train)
test = InvalidConvertor(test)

# 建立function 把原無效字元位置利用前後平均值取代
def FillByAvg(df):
    f = pd.DataFrame()
    b = pd.DataFrame()
    for col in df.columns:
        f[col] = df[col].fillna(method='ffill').astype('float')
        b[col] = df[col].fillna(method='bfill').astype('float')
    avg = (f + b) / 2
    return avg


train = FillByAvg(train)
test = FillByAvg(test)

# # 經由觀察剩下SO2最後一天的最後有15個空白，無法做平均，因此直接填入6.9
test = test.fillna('6.9')


In [8]:
test.isnull().sum()

AMB_TEMP      0
CH4           0
CO            0
NMHC          0
NO            0
NO2           0
NOx           0
O3            0
PM10          0
PM2.5         0
RAINFALL      0
RH            0
SO2           0
THC           0
WD_HR         0
WIND_DIREC    0
WIND_SPEED    0
WS_HR         0
dtype: int64

## (d)NR表示無降雨，以0取代


In [9]:
print(train['RAINFALL'].value_counts())  # 經由觀察未有NR值
test['RAINFALL'].value_counts() #經由觀察未有NR值

0.0    1355
0.1      72
0.2      16
0.4      11
0.8       5
0.6       4
2.4       1
Name: RAINFALL, dtype: int64


0.0    664
0.2     33
0.8     11
0.4     11
0.6      9
1.0      6
1.2      3
1.4      2
2.2      2
2.6      1
6.2      1
2.0      1
Name: RAINFALL, dtype: int64

## (e)製作時序資料: 將資料形式轉換為row代表18種屬性，欄(column)代表逐時數據資料


In [10]:
train_data = train.T
test_data = test.T
# 查看轉置後的訓練集資料
train_data


,0,1,2,3,4,5,6,7,8,9,...,1454,1455,1456,1457,1458,1459,1460,1461,1462,1463
AMB_TEMP,23.70,23.80,23.80,23.90,23.90,23.80,24.10,24.70,26.00,27.20,...,21.60,21.50,20.40,20.00,20.10,19.90,19.40,18.90,18.90,18.70
CH4,1.97,1.95,1.96,1.96,1.95,1.96,1.97,1.97,1.96,1.98,...,1.93,1.94,1.93,1.94,1.94,1.95,1.95,1.95,1.95,1.95
CO,0.23,0.22,0.21,0.20,0.20,0.22,0.24,0.29,0.27,0.33,...,0.26,0.27,0.27,0.29,0.29,0.31,0.25,0.22,0.20,0.18
NMHC,0.06,0.05,0.03,0.03,0.03,0.04,0.04,0.05,0.06,0.07,...,0.05,0.06,0.06,0.09,0.07,0.09,0.07,0.07,0.07,0.06
NO,1.20,0.70,0.50,0.70,0.50,0.30,0.70,0.90,1.00,1.80,...,2.50,2.40,2.00,1.80,1.60,1.60,1.80,1.70,1.60,1.60
NO2,8.00,6.00,5.50,5.20,5.30,5.80,8.00,7.60,6.60,8.00,...,4.50,5.40,6.60,9.00,7.50,8.60,6.90,6.00,4.80,4.10
NOx,9.20,6.70,6.10,5.80,5.80,6.30,8.60,8.50,7.60,9.80,...,6.90,7.70,8.50,10.80,9.10,10.30,8.70,7.80,6.30,5.70
O3,48.00,50.60,53.10,53.00,50.50,47.80,44.80,46.60,51.90,55.80,...,42.40,39.70,35.90,32.40,34.50,33.50,35.20,34.90,36.30,37.80
PM10,21.00,24.00,28.00,26.00,28.00,22.00,26.00,27.00,29.00,23.00,...,23.00,30.00,15.00,14.00,14.00,16.00,11.00,18.00,14.00,18.00
PM2.5,16.00,9.00,11.00,10.00,9.00,15.00,10.00,10.00,10.00,9.00,...,6.00,9.00,5.00,3.00,4.00,6.00,7.00,9.00,9.00,5.00


#  時間序列


## PM2.5

In [11]:
# PM2.5
# 未來第一個小時當預測目標
pm25_1hr_train_X = np.array([list(train_data.loc['PM2.5', i:i+5])
                            for i in range(len(train_data.columns)-6)])
pm25_1hr_train_y = np.array([train_data.loc['PM2.5', i+6]  # 5 + 1 預測未來一個小時
                            for i in range(len(train_data.columns)-6)])
pm25_1hr_test_X = np.array([list(test_data.loc['PM2.5', i:i+5])
                           for i in range(len(test_data.columns)-6)])
pm25_1hr_test_y = np.array([test_data.loc['PM2.5', i+6]
                           for i in range(len(test_data.columns)-6)])

# 未來第六個小時當預測目標
pm25_6hr_train_X = np.array([list(train_data.loc['PM2.5', i:i+5])
                            for i in range(len(train_data.columns)-11)])
pm25_6hr_train_y = np.array([train_data.loc['PM2.5', i+11]  # 5 + 6 預測未來6個小時
                            for i in range(len(train_data.columns)-11)])

pm25_6hr_test_X = np.array([list(test_data.loc['PM2.5', i:i+5])
                           for i in range(len(test_data.columns)-11)])
pm25_6hr_test_y = np.array([test_data.loc['PM2.5', i+11]
                           for i in range(len(test_data.columns)-11)])


## 所有屬性

In [12]:
# 所有屬性
# 將未來第一個小時當預測目標
all_1hr_train_X = np.array(list(np.array(train_data.loc[:,i:i+5]).ravel() for i in range(len(train_data.columns)-6)))
all_1hr_train_y = np.array([train_data.loc['PM2.5',i+6] for i in range(len(train_data.columns)-6)])

all_1hr_test_X = np.array(list(np.array(test_data.loc[:,i:i+5]).ravel() for i in range(len(test_data.columns)-6)))
all_1hr_test_y = np.array([test_data.loc['PM2.5',i+6] for i in range(len(test_data.columns)-6)])

# 將未來第六個小時當預測目標
all_6hr_train_X = np.array(list(np.array(train_data.loc[:,i:i+5]).ravel() for i in range(len(train_data.columns)-11)))
all_6hr_train_y = np.array([train_data.loc['PM2.5',i+11] for i in range(len(train_data.columns)-11)])

all_6hr_test_X = np.array(list(np.array(test_data.loc[:,i:i+5]).ravel() for i in range(len(test_data.columns)-11)))
all_6hr_test_y = np.array([test_data.loc['PM2.5',i+11] for i in range(len(test_data.columns)-11)])

## 訓練模型

In [13]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
other_params = {'eta': 0.1291549665014884, 'n_estimators': 100, 'gamma': 0, 'max_depth': 1, 'min_child_weight': 3,
                'colsample_bytree': 1, 'colsample_bylevel': 1, 'subsample': 1, 'reg_lambda': 60, 'reg_alpha': 3,
                'seed': 33}

cv_params = {'n_estimators': np.linspace(100, 1000, 10, dtype=int)}
XGB_all_1hr = XGBRegressor(**other_params)
gs = GridSearchCV(XGB_all_1hr, cv_params, verbose=2,
                  refit=True, cv=5, n_jobs=-1)
gs.fit(all_1hr_train_X, all_1hr_train_y)
print("參數的最佳取值: ", gs.best_params_)
print("最佳模型得分: ", gs.best_score_)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
參數的最佳取值:  {'n_estimators': 200}
最佳模型得分:  0.7009896104053952


In [14]:
other_params = {'eta': 0.1291549665014884, 'n_estimators': 200, 'gamma': 0, 'max_depth': 1, 'min_child_weight': 3,
                'colsample_bytree': 1, 'colsample_bylevel': 1, 'subsample': 1, 'reg_lambda': 60, 'reg_alpha': 3,
                'seed': 33}

cv_params = {'max_depth': np.linspace(1, 10, 10, dtype=int)}
XGB_all_1hr = XGBRegressor(**other_params)
gs = GridSearchCV(XGB_all_1hr, cv_params, verbose=2,
                  refit=True, cv=5, n_jobs=-1)
gs.fit(all_1hr_train_X, all_1hr_train_y)
print("參數的最佳取值: ", gs.best_params_)
print("最佳模型得分: ", gs.best_score_)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
參數的最佳取值:  {'max_depth': 1}
最佳模型得分:  0.7009896104053952


In [15]:
other_params = {'eta': 0.1291549665014884, 'n_estimators': 200, 'gamma': 0, 'max_depth': 1, 'min_child_weight': 3,
                'colsample_bytree': 1, 'colsample_bylevel': 1, 'subsample': 1, 'reg_lambda': 60, 'reg_alpha': 3,
                'seed': 33}

cv_params = {'min_child_weight': np.linspace(1, 10, 10, dtype=int)}
XGB_all_1hr = XGBRegressor(**other_params)
gs = GridSearchCV(XGB_all_1hr, cv_params, verbose=2,
                  refit=True, cv=5, n_jobs=-1)
gs.fit(all_1hr_train_X, all_1hr_train_y)
print("參數的最佳取值: ", gs.best_params_)
print("最佳模型得分: ", gs.best_score_)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
參數的最佳取值:  {'min_child_weight': 1}
最佳模型得分:  0.7009896104053952


In [16]:
other_params = {'eta': 0.1291549665014884, 'n_estimators': 200, 'gamma': 0, 'max_depth': 1, 'min_child_weight': 1,
                'colsample_bytree': 1, 'colsample_bylevel': 1, 'subsample': 1, 'reg_lambda': 60, 'reg_alpha': 3,
                'seed': 33}

cv_params = {'gamma': np.linspace(0, 0.1, 12, dtype=int)}
XGB_all_1hr = XGBRegressor(**other_params)
gs = GridSearchCV(XGB_all_1hr, cv_params, verbose=2,
                  refit=True, cv=5, n_jobs=-1)
gs.fit(all_1hr_train_X, all_1hr_train_y)
print("參數的最佳取值: ", gs.best_params_)
print("最佳模型得分: ", gs.best_score_)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
參數的最佳取值:  {'gamma': 0}
最佳模型得分:  0.7009896104053952


In [17]:
other_params = {'eta': 0.1291549665014884, 'n_estimators': 200, 'gamma': 0, 'max_depth': 1, 'min_child_weight': 1,
                'colsample_bytree': 1, 'colsample_bylevel': 1, 'subsample': 1, 'reg_lambda': 60, 'reg_alpha': 3,
                'seed': 33}

cv_params = {'subsample': np.linspace(0, 1, 11, dtype=int)}
XGB_all_1hr = XGBRegressor(**other_params)
gs = GridSearchCV(XGB_all_1hr, cv_params, verbose=2,
                  refit=True, cv=5, n_jobs=-1)
gs.fit(all_1hr_train_X, all_1hr_train_y)
print("參數的最佳取值: ", gs.best_params_)
print("最佳模型得分: ", gs.best_score_)


Fitting 5 folds for each of 11 candidates, totalling 55 fits
參數的最佳取值:  {'subsample': 1}
最佳模型得分:  0.7009896104053952


In [18]:
other_params = {'eta': 0.1291549665014884, 'n_estimators': 200, 'gamma': 0, 'max_depth': 1, 'min_child_weight': 1,
                'colsample_bytree': 1, 'colsample_bylevel': 1, 'subsample': 1, 'reg_lambda': 60, 'reg_alpha': 3,
                'seed': 33}


cv_params = {'colsample_bytree': np.linspace(0, 1, 11, dtype=int)}
XGB_all_1hr = XGBRegressor(**other_params)
gs = GridSearchCV(XGB_all_1hr, cv_params, verbose=2,
                  refit=True, cv=5, n_jobs=-1)
gs.fit(all_1hr_train_X, all_1hr_train_y)
print("參數的最佳取值: ", gs.best_params_)
print("最佳模型得分: ", gs.best_score_)


Fitting 5 folds for each of 11 candidates, totalling 55 fits
參數的最佳取值:  {'colsample_bytree': 1}
最佳模型得分:  0.7009896104053952


In [19]:
other_params = {'eta': 0.1291549665014884, 'n_estimators': 200, 'gamma': 0, 'max_depth': 1, 'min_child_weight': 1,
                'colsample_bytree': 1, 'colsample_bylevel': 1, 'subsample': 1, 'reg_lambda': 60, 'reg_alpha': 3,
                'seed': 33}


cv_params = {'reg_lambda': np.linspace(0, 100, 11, dtype=int)}
XGB_all_1hr = XGBRegressor(**other_params)
gs = GridSearchCV(XGB_all_1hr, cv_params, verbose=2,
                  refit=True, cv=5, n_jobs=-1)
gs.fit(all_1hr_train_X, all_1hr_train_y)
print("參數的最佳取值: ", gs.best_params_)
print("最佳模型得分: ", gs.best_score_)


Fitting 5 folds for each of 11 candidates, totalling 55 fits
參數的最佳取值:  {'reg_lambda': 60}
最佳模型得分:  0.7009896104053952


In [20]:
other_params = {'eta': 0.1291549665014884, 'n_estimators': 200, 'gamma': 0, 'max_depth': 1, 'min_child_weight': 1,
                'colsample_bytree': 1, 'colsample_bylevel': 1, 'subsample': 1, 'reg_lambda': 60, 'reg_alpha': 3,
                'seed': 33}


cv_params = {'reg_alpha': np.linspace(0, 10, 11, dtype=int)}
XGB_all_1hr = XGBRegressor(**other_params)
gs = GridSearchCV(XGB_all_1hr, cv_params, verbose=2,
                  refit=True, cv=5, n_jobs=-1)
gs.fit(all_1hr_train_X, all_1hr_train_y)
print("參數的最佳取值: ", gs.best_params_)
print("最佳模型得分: ", gs.best_score_)



Fitting 5 folds for each of 11 candidates, totalling 55 fits
參數的最佳取值:  {'reg_alpha': 4}
最佳模型得分:  0.7014321387790675


In [21]:
other_params = {'eta': 0.1291549665014884, 'n_estimators': 200, 'gamma': 0, 'max_depth': 1, 'min_child_weight': 1,
                'colsample_bytree': 1, 'colsample_bylevel': 1, 'subsample': 1, 'reg_lambda': 60, 'reg_alpha': 4,
                'seed': 33}

cv_params = {'eta': np.logspace(-2, 0, 10)}
XGB_all_1hr = XGBRegressor(**other_params)
gs = GridSearchCV(XGB_all_1hr, cv_params, verbose=2,
                  refit=True, cv=5, n_jobs=-1)
gs.fit(all_1hr_train_X, all_1hr_train_y)
print("參數的最佳取值: ", gs.best_params_)
print("最佳模型得分: ", gs.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
參數的最佳取值:  {'eta': 0.1291549665014884}
最佳模型得分:  0.7014321387790675


## Linear Regression

In [22]:
# Linear Regression - PM2.5
LR_pm25_1hr = LinearRegression(fit_intercept=False, normalize=True)
LR_pm25_1hr.fit(pm25_1hr_train_X, pm25_1hr_train_y)
LR_pm25_1hr_pred = LR_pm25_1hr.predict(pm25_1hr_test_X)
print(LR_pm25_1hr_pred)
print("MAE of Linear Regression - pm2.5 - 1hr PM2.5: ",
      mean_absolute_error(pm25_1hr_test_y, LR_pm25_1hr_pred))

LR_pm25_6hr = LinearRegression(
    fit_intercept=False, normalize=True)
LR_pm25_6hr.fit(pm25_6hr_train_X, pm25_6hr_train_y)
LR_pm25_6hr_pred = LR_pm25_6hr.predict(pm25_6hr_test_X)
print("MAE of Linear Regression - pm2.5 - 6hr PM2.5: ",
      mean_absolute_error(pm25_6hr_test_y, LR_pm25_6hr_pred))


# Linear Regression - 18 Attributes
LR_all_1hr = LinearRegression(
    fit_intercept=False, normalize=True)
LR_all_1hr.fit(all_1hr_train_X, all_1hr_train_y)
LR_all_1hr_pred = LR_all_1hr.predict(all_1hr_test_X)
print("MAE of Linear Regression - all - 1hr PM2.5: ",
      mean_absolute_error(all_1hr_test_y, LR_all_1hr_pred))

LR_all_6hr = LinearRegression(
    fit_intercept=False, normalize=True)
LR_all_6hr.fit(all_6hr_train_X, all_6hr_train_y)
LR_all_6hr_pred = LR_all_6hr.predict(all_6hr_test_X)
print("MAE of Linear Regression - all - 6hr PM2.5: ",
      mean_absolute_error(all_6hr_test_y, LR_all_6hr_pred))


[ 9.5194393   7.12851638  5.29173561  5.12458792  4.91330067  6.69093014
  5.93031595  8.84909508 10.45843782 11.78037135 10.70388432 10.52850982
  9.20609025  8.21898072 12.01396606  9.21479054  9.66791144  9.81177757
  9.56903321  9.12352595 11.12533169 12.21463294  9.25388394  7.4891362
  7.18598588  6.34182435  7.00346431  5.35661947  5.80736007  6.6836045
  8.19263464  9.28458513  7.05353108  5.37718356  5.9705077   8.29255741
  9.35558106  7.11642188 11.48406684  7.46524715  5.47866466  5.49932197
  6.5365519   6.98505022  8.27748348  6.15395065  4.35217839  6.59166031
  5.92558184  5.15417465  8.10878814  7.54168779  5.33344786  6.69391345
  6.70342385  6.0499695  11.39414238 11.11952654 10.45302916  9.88447809
 11.15995992  9.34045691  7.46571908  9.56219547  8.07734223  7.20892655
 10.1430411   7.22540963  5.50201664  3.77304944  5.58811006  5.2094943
  3.36190534  3.19466909  3.7564823   8.53441091  6.79503784  5.03381807
  5.80604724  5.73612995  6.75838601  7.5326228   6.77

## XGBoost

In [54]:
params_pm25_1hr = {'eta': 0.3, 'n_estimators': 100, 'gamma': 0, 'max_depth': 1, 'min_child_weight': 8,
                   'colsample_bytree': 1, 'colsample_bylevel': 1, 'subsample': 1, 'reg_lambda': 60, 'reg_alpha': 6,
                   'seed': 33}


params_pm25_6hr = {'eta': 0.0774263682681127, 'n_estimators': 100, 'gamma': 0, 'max_depth': 1, 'min_child_weight': 3,
                         'colsample_bytree': 1, 'colsample_bylevel': 1, 'subsample': 1, 'reg_lambda': 60, 'reg_alpha': 4,
                         'seed': 33}

params_all_1hr = {'eta': 0.1291549665014884, 'n_estimators': 200, 'gamma': 0, 'max_depth': 1, 'min_child_weight': 1,
                'colsample_bytree': 1, 'colsample_bylevel': 1, 'subsample': 1, 'reg_lambda': 60, 'reg_alpha': 4,
                'seed': 33}

params_all_6hr = {'eta': 0.046415888336127774, 'n_estimators': 100, 'gamma': 0, 'max_depth': 5, 'min_child_weight': 1,
                'colsample_bytree': 1, 'colsample_bylevel': 1, 'subsample': 1, 'reg_lambda': 10, 'reg_alpha': 6,
                'seed': 33}


In [55]:
# XGBoost
from xgboost import XGBRegressor
# XGBoost - PM2.5
XGB_pm25_1hr = XGBRegressor(**params_pm25_1hr)
XGB_pm25_1hr.fit(pm25_1hr_train_X, pm25_1hr_train_y)
XGB_pm25_1hr_pred = XGB_pm25_1hr.predict(pm25_1hr_test_X)
print("MAE of XGBoost - pm2.5 - 1hr PM2.5: ",
      mean_absolute_error(pm25_1hr_test_y, XGB_pm25_1hr_pred))

XGB_pm25_6hr = XGBRegressor(**params_pm25_6hr)
XGB_pm25_6hr.fit(pm25_6hr_train_X, pm25_6hr_train_y)
XGB_pm25_6hr_pred = XGB_pm25_6hr.predict(pm25_6hr_test_X)
print("MAE of XGBoost - pm2.5 - 6hr PM2.5: ",
      mean_absolute_error(pm25_6hr_test_y, XGB_pm25_6hr_pred))


# XGBoost - 18 Attributes

XGB_all_1hr = XGBRegressor(**params_all_1hr)
XGB_all_1hr.fit(all_1hr_train_X, all_1hr_train_y)
XGB_all_1hr_pred = XGB_all_1hr.predict(all_1hr_test_X)
print("MAE of XGBoost - all - 1hr PM2.5:",
      mean_absolute_error(all_1hr_test_y, XGB_all_1hr_pred))

XGB_all_6hr = XGBRegressor(**params_all_6hr)
XGB_all_6hr.fit(all_6hr_train_X, all_6hr_train_y)
XGB_all_6hr_pred = XGB_all_6hr.predict(all_6hr_test_X)
print("MAE of XGBoost - all - 6hr PM2.5: ",
      mean_absolute_error(all_6hr_test_y, XGB_all_6hr_pred))


MAE of XGBoost - pm2.5 - 1hr PM2.5:  2.654466922690229
MAE of XGBoost - pm2.5 - 6hr PM2.5:  4.770751389117065
MAE of XGBoost - all - 1hr PM2.5: 2.618156887974519
MAE of XGBoost - all - 6hr PM2.5:  4.436619846427262
